In [1]:
# ===================================================================
# STEP 1: IMPORT LIBRARIES
# ===================================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaModel
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ===================================================================
# STEP 2: LOAD TOKENIZER & MODEL (ROBERTA)
# ===================================================================
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
roberta = RobertaModel.from_pretrained(
    "roberta-base",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
).to(device)

print("✅ RoBERTa loaded successfully!")


# ===================================================================
# STEP 3: DATASET CLASS
# ===================================================================
class ComplaintDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=64):
        self.texts = dataframe["Complaint/ Opinion"].astype(str).tolist()
        self.labels = dataframe["Label"].tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoded = self.tokenizer(
            text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        return {
            "input_ids": encoded["input_ids"].squeeze(0),
            "attention_mask": encoded["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long),
        }


# ===================================================================
# STEP 4: BiGRU MODULE
# ===================================================================
class BiGRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(BiGRUModel, self).__init__()
        self.bigru = nn.GRU(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
        )

    def forward(self, x):
        output, hidden = self.bigru(x)
        return output, hidden  # output: [batch, seq_len, hidden*2]


# ===================================================================
# STEP 5: ATTENTION MODULE
# ===================================================================
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attention = nn.Linear(hidden_size * 2, 1)

    def forward(self, gru_output):
        scores = self.attention(gru_output)               # [batch, seq_len, 1]
        attn_weights = F.softmax(scores, dim=1)           # [batch, seq_len, 1]
        context_vector = torch.sum(attn_weights * gru_output, dim=1)  
        return context_vector, attn_weights               # [batch, hidden*2]


# ===================================================================
# STEP 6: CENTRAL BRAIN
# ===================================================================
class CentralBrain(nn.Module):
    def __init__(self, roberta_hidden_size=768, gru_hidden_size=128, num_classes=2):
        super(CentralBrain, self).__init__()
        self.central_fc = nn.Linear(roberta_hidden_size + gru_hidden_size * 2, 128)
        self.classifier = nn.Linear(128, num_classes)

    def forward(self, cls_embedding, attn_output):
        combined = torch.cat((cls_embedding, attn_output), dim=1)
        central_output = torch.relu(self.central_fc(combined))
        logits = self.classifier(central_output)
        return logits


# ===================================================================
# STEP 7: FINAL MODEL (CONNECT ALL PARTS)
# ===================================================================
class FullModel(nn.Module):
    def __init__(self, roberta, gru_hidden_size=128, num_classes=2):
        super(FullModel, self).__init__()
        self.roberta = roberta
        self.bigru = BiGRUModel(input_size=768, hidden_size=gru_hidden_size)
        self.attention = Attention(gru_hidden_size)
        self.central_brain = CentralBrain(768, gru_hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state    # [batch, seq_len, 768]
        cls_embedding = outputs.pooler_output          # [batch, 768]

        gru_output, _ = self.bigru(sequence_output)    # [batch, seq_len, hidden*2]
        attn_output, attn_weights = self.attention(gru_output)  # [batch, hidden*2]

        logits = self.central_brain(cls_embedding, attn_output) # [batch, num_classes]
        return logits


# ===================================================================
# STEP 8: TRAINING LOOP (with Mixed Precision)
# ===================================================================
def train_model(model, train_loader, epochs=3, lr=2e-5):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            optimizer.zero_grad()

            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                logits = model(input_ids, attention_mask)
                loss = criterion(logits, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")


# ===================================================================
# STEP 9: PUT EVERYTHING TOGETHER
# ===================================================================
df = pd.read_excel('C:\\Users\\BIT\\Downloads\\Complain.xlsx')
df.rename(columns={'Complaint Label':'Label'}, inplace=True)

dataset = ComplaintDataset(df, tokenizer, max_len=64)
train_loader = DataLoader(dataset, batch_size=8, shuffle=True)

model = FullModel(roberta, gru_hidden_size=128, num_classes=2).to(device)

# Freeze RoBERTa for faster training
for param in model.roberta.parameters():
    param.requires_grad = False

print("\n🚀 Starting training...")
train_model(model, train_loader, epochs=3, lr=2e-5)


Using device: cpu


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ RoBERTa loaded successfully!

🚀 Starting training...


C:\Users\BIT\AppData\Local\Temp\ipykernel_8784\2277501128.py:137: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())
C:\Users\BIT\AppData\Local\Temp\ipykernel_8784\2277501128.py:149: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):


Epoch 1/3, Loss: 0.5580
Epoch 2/3, Loss: 0.2431
Epoch 3/3, Loss: 0.2001


In [12]:
# ===================================================================
# STEP 10: INFERENCE ON NEW TEXT
# ===================================================================
model.eval()  # put model in evaluation mode

new_text = "Unauthorized transactions appeared on my debit card"

# Tokenize
encoded = tokenizer(
    new_text,
    max_length=64,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

input_ids = encoded["input_ids"].to(device)
attention_mask = encoded["attention_mask"].to(device)

# Get prediction
with torch.no_grad():
    logits = model(input_ids, attention_mask)
    probs = torch.softmax(logits, dim=1)
    predicted_class = torch.argmax(probs, dim=1).item()

print(f"Input: {new_text}")
print(f"Prediction class: {predicted_class}")
print(f"Probabilities: {probs.cpu().numpy()}")


Input: Unauthorized transactions appeared on my debit card
Prediction class: 1
Probabilities: [[0.22745936 0.7725406 ]]


In [10]:
import os

save_dir = "saved_model"
os.makedirs(save_dir, exist_ok=True)
torch.save(model.state_dict(), os.path.join(save_dir, "full_model.pth"))
tokenizer.save_pretrained(save_dir)

print("✅ Model and tokenizer saved!")


✅ Model and tokenizer saved!


In [11]:
tokenizer = RobertaTokenizer.from_pretrained("saved_model")
loaded_model = FullModel(roberta, gru_hidden_size=128, num_classes=2).to(device)
loaded_model.load_state_dict(torch.load("saved_model/full_model.pth", map_location=device))
loaded_model.eval()

print("✅ Model and tokenizer loaded successfully!")


✅ Model and tokenizer loaded successfully!
